## Calculations for jacobian from Graph slamm tutorial paper

According to the previous explanation we have:

$X_{i}^{-1}X_{j} = \begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}$

Inversion of a block transformation matrix can be expresed with the following formula:

$\begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}^{-1} = \begin{bmatrix} \begin{bmatrix} I & t\\ 0 & 1 \end{bmatrix} & \begin{bmatrix} R & 0 \\ 0 & 1 \end{bmatrix} \end{bmatrix}^{-1} = \begin{bmatrix} \begin{bmatrix} R & 0 \\ 0 & 1 \end{bmatrix}^ {-1} & \begin{bmatrix} I & t\\ 0 & 1 \end{bmatrix}^{-1} \end{bmatrix}=\begin{bmatrix} \begin{bmatrix} R^{T} & 0\\ 0 & 1 \end{bmatrix}&\begin{bmatrix} I & -t\\ 0 & 1 \end{bmatrix} \end{bmatrix}= \begin{bmatrix} R^{T} & -R^{T}t\\ 0 & 1 \end{bmatrix}$

So we can rewrite the error function in the following form:

$Z_{ij}^{-1}(X_{i}^{-1}X_{j}) = \begin{bmatrix} R' & t'\\ 0 & 1 \end{bmatrix}^{-1} \begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}=\begin{bmatrix} R'^{T} & -R't'\\ 0 & 1 \end{bmatrix}\begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}=\begin{bmatrix} R'^{T}R & Rt-R't'\\ 0 & 1 \end{bmatrix}=\begin{bmatrix}R'^{T}R & R'^{T}(t-t')\\0 & 1\end{bmatrix}$

And the same function in the vectorized form:

$e_{ij}(x_i,x_j)=Z_{ij}^{-1}(\begin{bmatrix}R_i^{T}(t_j-t_i)\\ \theta_j - \theta_i \end{bmatrix})$

$\Delta t_{ij} = R_{z}^{T}\begin{bmatrix} R_{i}^{T}(\begin{bmatrix} x_j \\ y_j\end{bmatrix} - \begin{bmatrix} x_i \\ y_i\end{bmatrix}) - \begin{bmatrix} x_z \\ y_z\end{bmatrix})\end{bmatrix}$

$\Delta \theta_{ij} = (\theta_j - \theta_i) - \theta_z$


The form of the $X$ vector for the optimization problem - is one long vector stacked from all pose vectors:

$x =[(x_1, y_1, \theta_1), (x_2, y_2, \theta_2), ...., (x_n, y_n, \theta_n)]$

Our error term doesn't depend on all state variables but on only on 3 values blocks related to the $i$-th and $j$-th poses.

And this fact is also visible in the form of Jacobian:

$J_{ij} = \begin{bmatrix} 0 & ... & 0 & A_{ij} &  0 & ... & 0 & B_{ij} & 0 & ... & 0 \end{bmatrix}$

$A_{ij} = \frac{\partial e_{ij}(x_i)}{\partial x_i}$

$B_{ij} = \frac{\partial e_{ij}(x_j)}{\partial x_j}$

So for $A_{ij}$ and $B_{ij}$ we have the following formulas:

$A_{ij} = \begin{bmatrix} -R_{z}^T R_{i}^T &  R_{z}^T \frac{\partial R_{i}^T}{\partial \theta_i} (t_j - t_i) \\ 0 & -1 \end{bmatrix}$

where:

1. the 3x2 block $\begin{bmatrix}-R_{z}^T R_{i}^T \\ 0 \end{bmatrix}$ is the partial derrivative of $\Delta t_{ij}$ in the top and  $\Delta \theta_{ij}$ in the bottom w.r.t $\left[x_i y_i\right]$

2. the 3x1 block $\begin{bmatrix} R_{z}^T \frac{\partial R_{i}^T}{\partial \theta_i} (t_j - t_i) \\ -1 \end{bmatrix}$ is the partial derrivative of $\Delta t_{ij}$ in the top and  $\Delta \theta_{ij}$ in the bottom w.r.t $\theta_i$

3. $\frac{\partial R_{i}^T}{\partial \theta_i} = \begin{bmatrix} -sin(\theta_i) & cos(\theta_i) \\  -cos(\theta_i) & -sin(\theta_i) \end{bmatrix}$

In the same manner we take partial derrivatives w.r.t $\left[x_j y_j\right]$ and $\theta_j$:


$B_{ij} = \begin{bmatrix} R_{z}^T R_{i}^T &  0 \\ 0 & 1 \end{bmatrix}$

In [ ]:
import numpy as np

si = np.sin(v_i[2])
ci = np.cos(v_i[2])
dr_i = np.array([[-si, ci], [-ci, -si]]).T
dt_ij = np.array([v_j[:2] - v_i[:2]]).T

t_i = v2t(v_i)
t_j = v2t(v_j)
r_i = t_i[:2,:2]
r_z = t_z[:2,:2]

a_ij = np.vstack((np.hstack((-r_z.T @ r_i.T, (r_z.T @ dr_i.T) @ dt_ij)), 
                         [0, 0, -1]))
print(f'The shape of A_ij is {a_ij.shape}')

b_ij = np.vstack((np.hstack((r_z.T @ r_i.T, np.zeros((2,1)))),
                         [0, 0, 1]))
print(f'The shape of B_ij is {b_ij.shape}')

In [12]:

class Frame:
    def __init__(self, observed_points):
        self.position = np.array([0., 0., 0.])
        self.rotation = np.identity(3)
        self.relative_icp_position = np.array([0., 0., 0.])  # relative to the previous frame
        self.relative_icp_rotation = np.identity(3)  # relative to the previous frame
frame= Frame([])
print(frame.rotation)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [13]:
rot = frame.rotation[:2]

In [14]:
print(rot)

[[1. 0. 0.]
 [0. 1. 0.]]


In [7]:
tr = np.identity(3)
relative_icp_rotation = np.identity(3)
relative_icp_position = np.array([0., 0., 0.])
# relative_icp_rotation = np.array([0., 0., 0.]) 
tr[:, :] = relative_icp_rotation
tr[:2, 2] = relative_icp_position[:2]

In [8]:
print(tr)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [10]:
relative_icp_position[:2]

array([0., 0.])